In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import openmc

import os
import numpy as np

In [ ]:
#Initial starting source, uniform in fuel only
bounds = [-0.4, -0.4, -0.4, 0.4, 0.4, 0.4]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)

In [ ]:
settings = openmc.Settings()
settings.source = openmc.source.Source(space=uniform_dist)
settings.batches = 100
settings.inactive = 25  # keep this at a min of 25, this is the number of power iterations performed, more is always better
settings.particles = 1000  # increase this number to make your results more accurate
settings.temperature = {'tolerance':10000,'multipole':True}
#settings.temperature = {'method':'interpolation','multipole':True}

In [ ]:
settings.export_to_xml()

In [ ]:
pitch = [1.05, 1.25, 1.45]
boron = [0]
nt = len(pitch)
nb = len(boron)
temp1 = 900  #fuel temp
temp2 = 600  #mod temp
k = np.zeros([nt,nb])
for m in range(nb):
    for j in range(nt):
        uo2 = openmc.Material(1,"fuel",temperature=temp1)
        uo2.add_element('U', 1.0, enrichment=4.0)
        uo2.add_element('O', 2.0)
        uo2.set_density('g/cc', 10.0)
        zirconium = openmc.Material(2, "zirconium", temperature=temp2)
        zirconium.add_element('Zr', 1.0)
        zirconium.set_density('g/cm3', 6.6)
        #this function creates borated light water, for other moderators you will need to replace
        water = openmc.model.borated_water(boron_ppm=boron[m], temperature=600,pressure=15)
        mats = openmc.Materials([uo2, zirconium, water])
        mats.export_to_xml()
        fuel_or = openmc.ZCylinder(R=0.39)
        clad_ir = openmc.ZCylinder(R=0.40)
        clad_or = openmc.ZCylinder(R=0.46)
        fuel_region = -fuel_or
        gap_region = +fuel_or & -clad_ir
        clad_region = +clad_ir & -clad_or
        fuel = openmc.Cell(1, 'fuel')
        fuel.fill = uo2
        fuel.region = fuel_region
        gap = openmc.Cell(2, 'air gap')
        gap.region = gap_region
        clad = openmc.Cell(3, 'clad')
        clad.fill = zirconium
        clad.region = clad_region
        box = openmc.get_rectangular_prism(width=pitch[j], height=pitch[j],
                                   boundary_type='reflective')
        water_region = box & +clad_or
        moderator = openmc.Cell(4, 'moderator')
        moderator.fill = water
        moderator.region = water_region
        root = openmc.Universe(cells=(fuel, gap, clad, moderator))
        geom = openmc.Geometry(root)
        geom.export_to_xml()
        cell_filter = openmc.CellFilter([fuel,gap, clad, moderator])
        #tallies over 2 energy groups with 4 eV being thermal bound
        energy_filter = openmc.EnergyFilter([0., 4.0, 20.0e6])
        t = openmc.Tally(1)
        t.filters = [cell_filter, energy_filter]
        # these are the main reaction rates you should need
        t.scores = ['absorption','nu-fission','fission']
        tallies = openmc.Tallies([t])
        tallies.export_to_xml()
        openmc.run()
        #make sure the number in this file name matches the number of cycles you are running!!!
        sp = openmc.StatePoint('statepoint.100.h5')
        # this reads the tally with all reaction rates, not just absorption
        tally = sp.get_tally(scores=['absorption'])
        # this is the final k-effective of the simulation
        k[j,m] = sp.k_combined[0]
        os.remove('statepoint.100.h5')
        os.remove('summary.h5')
        del sp

In [ ]:
print(k)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(pitch,k[:,0], linewidth=10)
plt.legend(['0ppm Boron'], fontsize=30)
plt.xlabel('Pitch', fontsize=30)
plt.ylabel('k', fontsize=30)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)